# Numerical Computation

Machine Learning involves numerical computations that are used to solve mathematical problems via an iterative process that updates the estimates of the solution until some convergence criterion is reached. This is contrary to analytically determining a formula to provide a symbol expression for the correct solution which is a common approach in both engineering and mathematics. Common numerical operations used in ML are:

1. Optimization
2. Solving a system of linear equations

## 4.1 Overflow and Underflow

On a computer, we use a finite number of bit patterns (due to a finite amount of available memory) to represent infinitely many real numbers. This causes us to make approximations that can lead to rounding errors and can even cause a theoretically proven algorithm to fail in practice.

__Underflow__ is one such form of rounding error and occurs when a number near zero gets rounded off to zero. This can cause operations such as "divide by" or "log" to fail since both operations are undefined for 0.

__Overflow__ on the other hand occurs when numbers with large magnitudes are approximated as $\inf$ or $-\inf$. 

The __softmax function__ $softmax(\pmb{x})_i = \dfrac{\exp(x_i)}{\sum_{j = 1}^{n}\exp(x_j)}$ for example, must be stabilized against underflow and overflow. Let's take a look at this example:

In [23]:
import math
def softmax(x, i):
    return math.exp(x[i]) / sum(map(lambda v: math.exp(v), x))

xs = [[1] * 10, [math.pow(2, 32)] * 10, [-math.pow(2, 32)] * 10]

# softmax without exponent
for x in xs:
    try: 
        print(softmax(x, 1))
    except OverflowError:
        print(f'Overflow error for an array with all values = {x[0]}')
    except ZeroDivisionError:
        print(f'Divison By Zero error for an array with all values = {x[0]}')

# Normalize the values in x before taking the exponent
for x in xs:
    try:
        m = max(x)
        zs = list(map(lambda v: v - m, x))
        print(softmax(zs, 1))
    except OverflowError:
        print(f'Overflow error for an array with all values = {x[0]}')
    except ZeroDivisonError:
        print(f'Divison By Zero error for an array with all values = {x[0]}')

0.1
Overflow error for an array with all values = 4294967296.0
Divison By Zero error for an array with all values = -4294967296.0
0.1
0.1
0.1


We can clearly see from the the example above that when $\forall i, x_i = c$ and x has length n then we expect the value of the softmax to be $\dfrac{1}{n}$. But that is not what happens when c is very large or small. To maneuver against this what we do is modify each element in the array, xs using the following operation $\forall i, z_i = x_i - max_j x_j$ and then use the resulting zs to calculate softmax. This maneuvering does not alter the expected output and also allows us to tackle the issues of overflow and division by zero.

However, if our numerator $x_i$ is a large negative number and all the other numbers in the list of values are positive. The numerator of softmax might get round off to zero resulting in underflow. Moreover, if we take $\log(softmax(\pmb{x})$ then we would end up with $-\inf$. But to counteract this we make use of log rules in evaluating $\log(softmax(x)_i)$. We thus present a completely stabilized version of $\log(softmax(x)_i)$ below:

$$
\log(softmax(x)_i) = \log\left(\dfrac{\exp(x_i)}{\sum_{j = 1}^{n}\exp(x_j)}\right) \\
= \log\left(\dfrac{\exp(x_i - b)\exp(b)}{\sum_{j = 1}^{n}\exp(x_j - b)\exp(b)}\right) \\ 
= \log\left(\exp(x_i - b)\right) - \log\left(\sum_j \exp(x_j - b)\right) \\
= (x_i - b) - \log\left(\sum_j \exp(x_j - b)\right)
$$

## 4.2 Poor Conditioning

Conditioning refers to how rapidly a function changes with respect to small changes in its input. When a function changes rapidly with small changes to its input it can be problematic for scientific computations because of rounding errors. 

Let's consider $f(\pmb x) = A^{-1} \pmb{x}$ assuming $A$ has an eigenvalue decomposition. The __condition number__ for $A$ is:

$$
\max_{i, j} \left|\dfrac{\lambda_i}{\lambda_j}\right|
$$

which is the ratio of the largest and smallest eigenvalues. When this number is large matrix inversion is particularly sensitive to error in inputs. This sensitivity is intrinsic to the matrix itself not the result of rounding error during matrix inversion. Poorly conditioned matrices amplify pre-existing errors when we multiply by the true matrix inverse.


## 4.3 Gradient-Based Optimization

Most deep learning algorithms involve optimization of some sort. Optimization refers to the task of minimizing or maximizing some function $f(x)$ by changing $x$. Here $f$ is the __objective function__ or __criterion__. In deep learning we frame problems as minimizing $f(x)$. During minimization $f$ might also be called the __cost function__, __loss function__ or __error function__. $x^*$ is the value that minimizes the function $f$, so $x^* = arg min f(x)$.

Let's say $y = f(x)$. The __derivative__ of this function is denoted by $f'(x)$ or as $\dfrac{dy}{dx}$. The derivative gives the slope of $f$ at $x$. It specifies how to scale a small change in the input to obtain a corresponding change in the output: $f(x + \epsilon) \approx f(x) + \epsilon f'(x)$.

The derivative is used for minimizing a function because it tells us how to change $x$ to make a small improvement in $y$. We can decrease $f(x)$ by moving $x$ in small steps in the direction with the opposite sign to the derivative. This technique is called __gradient descent__.


<img src="images/04/gradient-descent.png" width="600"/>

When $f'(x) = 0$ the derivative gives us no information about the direction we should move in. These points where the derivative is 0 are known as __critical points__ or __stationary points__. We can classify them as:

1. Local minimum - a point where $f(x)$ is lower than all neighboring points. 
2. Local maximum - a points where $f(x)$ is higher than all neighboring points.
3. Saddle Points - a point where $f(x)$ is neither a maximum or a minimum.

<img src="images/04/stationary-points.png" width="600"/>

A point that obtains the lowest value of $f(x)$ across all possible $x$ is called the __global minimum__. A local minimum does not have to be a global minimum. In deep learning we usually settle for finding a value that is very low but not necessarily  minimal in any sense due to the complex nature of functions being minimized.

We often minimize function that are defined as $f: \mathbb{R}^n \to \mathbb{R}$. For minimization to make sense the output must be a scalar. In case of multiple inputs we must take __partial derivatives__. The partial derivative $\dfrac{\partial f(\pmb{x})}{\partial x_i}$ measures what direction we should move $x_i$ in to be able to increase $f$ all other variables staying the same. The gradient of a function with respect to it's input is denoted as $\bigtriangledown_{\pmb{x}}f(\pmb{x})$ where element i of this gradient corresponds to differentiation with respect to $x_i$.

The __directional derivative__ in the direction ___u___ (where __u__ a unit vector) is the slope of $f$ in the direction $u$. The directional derivative is the derivative of the function $f(x + \alpha u)$ with respect to $\alpha$, which evaluates to $\pmb{u}^T\bigtriangledown_{\pmb x} f(\pmb x)$ when $\alpha = 0$.

To minimize $f$ we would like to find the direction in which $f$ decreases fastest. We can do this using the directional derivative:

$$
min_{u, u^T u = 1} u^T \bigtriangledown_x f(x) \\
min_{u, u^T u = 1} ||u||_2 ||\bigtriangledown_x f(x)||_2 \cos(\theta) \\
$$

where $\theta$ is the angle between $u$ and the gradient. Now, since u has a magnitude of 1 and the magnitude of the gradient does not depend on u. The equation above boils down to minimizing $\cos(\theta)$ which is minimized when $\cos(\theta) = -1$ and u is opposite to the direction of the gradient. This is known as __method of steepest descent__ or __gradient descent__. Thus, steepest descent proposes a new points:

$$
x' = x - \epsilon \bigtriangledown_x f(x)
$$

where \epsilon is the __learning rate__, a small positive scalar determining the size of our step in the direction of steepest descent. Sometimes we can solve for a step size that makes the directional derivative vanish. Another approach is to evaluate $f(x - \epsilon \bigtriangledown_x f(x))$ for several values of $\epsilon$ and choose the one that results in the smallest value of the objective function. This process is known as __line search__.

Steepest descent converges when every element of the gradient is zero. We can adapt the idea of making small steps to discrete spaces which is a process called __hill climbing__.

## Beyond the Gradient: Jacobian and Hessian Matrices

The __Jacboian Matrix__ corresponds to the matrix containing all the partial derivatives (with respect to the input) of a function whose output is also multi-dimensional, $f: \mathbb{R}^m \to \mathbb{R}^n$. So, the Jacboian, $J \in \mathbb{R}^{n \times m}$ of $f$ is defined such that $J_{i, j} = \dfrac{\partial f(x)_i}{\partial x_j}$

The __second derivative__ (derivative of the derivative) for a function, $f: \mathbb{R}^n \to \mathbb{R}$ is given by $\dfrac{\partial^2 f}{\partial x_i \partial x_j}$  which corresponds to taking the derivative with respect to $x_i$ of $\dfrac{\partial f}{\partial x_j}$. In a single dimension the second derivative is denoted by $f''(x)$. In the case of multiple input dimensions, there are many second derivatives and we denote them using the __Hessian Matrix__. The Hessian matrix $H(f))(x)$ is defined by:

$$
H(f)(x)_{i, j} = \dfrac{\partial^2 f(x)}{\partial x_i \partial x_j}
$$

The Hessian is the Jacobian of the gradient. __Note__: The Hessian is the derivative of the gradient and not the Jacobian (the derivative of the Jacobian would be a 3-D tensor).

The second derivative tells us how the first derivative will change as we vary the input. We can think of it as measuring the __curvature__ of the function. 

The diagram below gives us an idea how the second derivative gives us information about the curvature of a function:

<img src="images/04/second-derivative.png" width="600"/>

Anywhere the second partial derivative are continuous, the differential operators are commutative:

$$
\dfrac{\partial^2 f(x)}{\partial x_i \partial x_j} = \dfrac{\partial^2 f(x)}{\partial x_j \partial x_i}
$$

This makes the Hessian matrix symmetric at such points. Most of the functions we encounter in the context of deep learning have a symmetric Hessian almost anywhere. Because the Hessian is real and symmetric we can decompose it into a set of real eigenvalues and an orthogonal basis of eigenvectors. The second derivative in a specific direction represented by a unit vector $d$ is given by $d^T H d$. When $d$ is an eigenvector of $H$ the second derivative in that direction is given by the corresponding eigenvalue. In other directions it is a weighted average of the eigenvalues (weights between 0 and 1).

We can make a second order Taylor Approximation of the function f(x) around the current point $x^{(0)}$:

$$
f(x) \approx f(x^{(0)}) + (x - x^{(0)})g + \dfrac{1}{2}(x - x^{(0)})^T H (x - x^{(0)})
$$

The terms of the equation above are: the original value of the function, the gradient of the function and the correction we must apply to the gradient to account for the curvature of the function. Putting the point $x^{(0)} - \epsilon g$

$$
f(x) \approx f(x^{(0)}) - \epsilon g^T g + \dfrac{1}{2}\epsilon^2 g^T H g
$$

When the last term is too large the gradient descent can actually move uphill. When the last term is 0 then gradient descent predicts that increasing $\epsilon$ forever will decrease $f$ forever. The Taylor Series approximation us unlikely to stay accurate for large $\epsilon$ so we resort to heuristic choices of $\epsilon$. When $g^THg$ is positive solving for the optimal step size that decreases the Taylor series approximation of the function yields:

$$
\epsilon^* = \dfrac{g^T g}{g^T H g}
$$

In the worst case $g$ aligns with the eigenvector that corresponds to the maximum value of $\lambda_{max}$, then the optimal size is given by $\dfrac{1}{\lambda_{max}}$. Depending upon how well the 2nd order Taylor approximation works for a function we can use the eigenvalues of the Hessian to determine the scale of the learning rate.

We can use the second derivative to figure out the nature of a stationary points for a function, $f: \mathbb{R} \to \mathbb{R}$. If $f'(x) = 0$ then:

* $f''(x) > 0$ then the stationary point is a local minimum.
* $f''(x) < 0$ then the stationary point is a local maximum.
* $f''(x) = 0$ then the stationary point is a saddle point or part of a flat region.

The above test is known as the __second derivative test__. 

Similarly, for $f: \mathbb{R}^n \to \mathbb{r}$ we use the Hessian to figure out the nature of a stationary point:

* If the Hessian is positive definite the point is a local minimum. 
* If the Hessian is negative definite the point is a local maximum. 
* If the Hessian has one positive eigenvalue and one negative eigenvalue the we know we have a saddle point.
* The test is inconclusive for positive semi-definite and negative semi-definite matrices.

<img src="images/04/saddle-point.png" />

In multiple dimensions, there is a different second derivative for each direction at a single point. The condition number of the Hessian matrix at this point measures how much the second derivatives differ from each other. When the Hessian has a poor condition number, gradient descent performs poorly because in one direction the derivative increases rapidly, while in another direction, it increases slowly. Gradient descent is unaware of this change in the derivative so it does not know that it needs to explore preferentially in the direction where the derivative remains negative for longer. 

To solve this problem we use information from the Hessian Matrix to guide the search. __Newton's Method__ is based on the second order Taylor expansion of $f(x)$ at $x^{(0)}$:

$$
f(x) \approx f(x^{(0)}) + (x - x^{(0)})^T\bigtriangledown_x f(x^{(0)}) + \dfrac{1}{2}(x - x^{(0)})^T H(f)(x^{(0)}) (x - x^{(0)})
$$

Differentiating with respect to $x$ and setting it equal to 0 (in order to get a stationary points) we get:

$$
0 = \bigtriangledown_x f(x^{(0)}) + (x - x^{(0)}) H(f)(x^{(0)}) \\
\bigtriangledown_x f(x^{(0)}) = (x - x^{(0)}) H(f)(x^{(0)}) \\
\bigtriangledown_x f(x^{(0)}) + x^{(0)}H(f)(x^{(0)}) = x H(f)(x^{(0)}) \\
x  = H^{-1}(f)(x^{(0)}) \bigtriangledown_x f(x^{(0)}) + x^{(0)} 
$$



When $f$ is a positive definite quadratic function, Newton's method consists of applying the equation above to jump straight to the minimum. If $f$ is not quadratic but can be approximated as such then Newton's method turns into an iterative process. But if we don't know that $f$ is positive-definite then Newton's method might be attracted to a saddle point when applied near the saddle point whereas gradient descent is not attracted to saddle points.

Optimization algorithms that use only the gradient are known as __first-order optimization algorithms__. Algorithms that use the Hessian as well are known as __second-order optimization algorithms__. 

Most of the optimization algorithms employed in deep learning come without any guarantees because the family of functions used in deep learning is quite complicated. We sometimes get some guarantees by restricting ourselves to functions that are __Lipschitz continuous__ or have Lipschitz continuous derivatives. A Lipschitz continuous function is a function, $f$ whose rate of change is bounded by a __Lipschitz constant__ $\mathcal{L}$:

$$
\forall x \forall y |f(x) = f(y)| \le \mathcal{L}|| x - y ||_2
$$

This property is useful because it helps us quantify that a small change in the input will result in a small change in the output. __Convex Optimization__ is a field of specialized optimization wherein the functions we perform optimization on need to be convex - have a positive semi-definite Hessian. This allows such function to be extremely well-behaved because they don't have saddle points and all their local minimas are necessarily global minimas. But the functions used in deep learning are rarely so well-behaved but ideas from convex analysis have proved useful for proving convergence of deep learning algorithms.

## 4.4 Constrained Optimization

__Contrained Optimization__ refers the scenario where we want to optimize the value of a function $f(x)$ given that $x$ lies in some set $\mathbb{S}$. Points $x$ that lie within 